# Scenario 1 - Kinetic Modeling
This Jupyter notebook will guide you through the process of visualizing and modeling your experimental data and finally uploading an EnzymeML document to a Database. The EnzymeML data format is based on the Systems Biology Markup Language (SBML) and helps with data consistency by storing all experimental data and the corresponding metadata in one place. <br>
A Juypter notebook contains text-cells, like this one, and code-cells, which you can run by pressing the small triangle on the top of the page or with the shortcut (CTRL+Return). Some code-cells will generate output directly under them. The current cell is highlighted by a blue line on the left. To choose a specific cell just click it. <br>
You can use this notebook as a step-by-step tutorial. For each step, a short text explains what will happen when you run the code-block and what you have to do. <br>
If not stated otherwise, please run each code-cell after you read the corresponding text.<br>
Before we start, we have to do some preparations in the next few cells. 

### Packages imports
Import packages used in this notebook

In [2]:
# API
import requests
import json
# Operating system for files
import os
# Jupyter widgets
from ipywidgets import widgets
# HTML
from IPython.core.display import display, HTML

### Working directory
To have one place for all your data choose a directory. When you press the button a directory manager will open and you can navigate to a folder of your choice.

### Preparing the API
For convenient API-requests later on, some preparation has to be done.

In [ ]:
url = "http://127.0.0.1:5000/"
endpoint_create = f"{url}/create"
endpoint_read = f"{url}/read"

## 1) EzymeML spreadsheet with experimental data
For your convenience, an EnzymeML spreadsheet template was designed, including tooltips for additional information. You can download a zip containing the empty Excel spreadsheet and an example of a filled spreadsheet [here](https://www.dropbox.com/s/ap26t13upye3cte/EnzymeML_Template_V1.zip?dl=1). <br> 
Please fill in the spreadsheet with your data and save it to the previously chosen directory. <br> 
If you have further questions or remarks, please contact us.

## 2) Convert the EnzymeML spreadsheet to an EnzymeML document
In this step the EnzymeML spreadsheet will be converted to an EnzymeML document. <br>
The EnzymeML document which is generated upon conversion is (1) a local .omex file or (2) a new entry in an EnzymeML dataverse <br>

2.1) Convert the EnzymeML spreadsheet to a local EnzymeML file in .omex format<br>
Run the next code cell to convert the spreadsheet to an EnzymeML document. An empty field and a button will appear. Enter the filename for the EnzymeML file in the empty field. Then press the button, from the window select the spreadsheet, and confirm. Now the spreadsheet will be automatically converted to an EnzymeML document via an API request. You can find the EnzymeML document with the file ending .omex in your working directory.<br>

2.2) Convert the EnzymeML spreadsheet to an EnzymeML dataverse <br>
Run the next code cell to convert the spreadsheet to an EnzymeML document. An empty field and a button will appear. Enter the address of the EnzymeML dataverse in the empty field. Then press the button, from the window select the spreadsheet, and confirm. Now the spreadsheet will be automatically converted to an EnzymeML document via an API request. You can find the EnzymeML document as new EnzymeML dataverse entry.<br>

## 3) Visualizing of time-course data
The next code cell will give you a static basic overview of your time-course data. Later on, you have the chance to dive deeper into your data with interactions.

## 4) Kinetic modeling
Kinetic modeling consists of two steps: (1) Add a kinetic model to the EnzymeML document (2) Perform a parameter estimation using the selected kinetic model and the selected time course data in the EnzymeML document.<br>

4.1) Add a kinetic model to the EnzymeML document
Run the code cell with your selected model. You might add more than one model to the EnzymeML document. For consistency, we use [Systems Biology Ontology](http://www.ebi.ac.uk/sbo/main/) (SBO) Terms.

Irreversible Michaelis-Menten
[SBO:000029](http://www.ebi.ac.uk/sbo/main/SBO:0000029):

Michaelis-Menten with product inhibition
[SBO:0000387](http://www.ebi.ac.uk/sbo/main/SBO:0000387):

Reversible Michaelis-Menten
[SBO:0000438](http://www.ebi.ac.uk/sbo/main/SBO:0000438):

4.2) Perform a parameter estimation using the selected kinetic model and the selected time course data in the EnzymeML document <br>
Select the model and the time course data to be modeled, then use the COPASI API.

## 5) Visualizing the results
Visualize the kinetic model(s) from the previous step and compare the results.

## 6) Create a DataVerse entry
Run the next code cell to make a dataverse entry. A dropdown menu and a button will appear. Choose a dataverse. Then press the button, from the window select the EnzymeML document, and confirm. Now the EnzymeML document will be automatically transmitted via an API request. You can find the EnzymeML document as a new EnzymeML dataverse entry.<br>

## 7) Publish to SABIO-RK
Finally, publish your data to the SABIO-RK database. Press the button and select your EnzymeML document. After your confirmation, the EnzymeML document will automatically be validated and uploaded to the SABIO-RK database.